In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
project_name = "google_colab"
git_repo = f"https://github.com/fmaclean/{project_name}.git"
use_conda = True

In [3]:
!nvidia-smi

Sat Jan 16 23:51:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install tqdm
!rm -rf sample_data
!rm -rf $project_name
!git clone $git_repo

Cloning into 'google_colab'...
remote: Enumerating objects: 22732, done.
remote: Counting objects: 100% (22732/22732), done.
remote: Compressing objects: 100% (15496/15496), done.
remote: Total 22732 (delta 6323), reused 22706 (delta 6303), pack-reused 0
Receiving objects: 100% (22732/22732), 45.94 MiB | 22.64 MiB/s, done.
Resolving deltas: 100% (6323/6323), done.


In [5]:
input_files = ['data__external__drkg__.zip']
for input_file in input_files:
  input_path = input_file.replace('__','/').replace('.zip','')[:-1]
  input_path = '/'.join(input_path.split('/')[:-1])
  !rm -rf $project_name/$input_path
  !mkdir -p $project_name/$input_path
  !unzip drive/MyDrive/$project_name/$input_file -d $project_name/$input_path/

Archive:  drive/MyDrive/google_colab/data__external__drkg__.zip
  inflating: google_colab/data/external/drkg/relation_glossary.tsv  
  inflating: google_colab/data/external/drkg/drkg.tsv  
  inflating: google_colab/data/external/drkg/.DS_Store  
  inflating: google_colab/data/external/drkg/entity2src.tsv  


In [7]:
if use_conda:
  !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
  !conda update conda -y -q
  !source /usr/local/etc/profile.d/conda.sh
  !conda init 
  !conda install -n root _license -y -q
  !conda env create -f /content/$project_name/environment.yml
  !source activate $project_name && conda install -c rdkit rdkit -y 
  !cd $project_name && source activate $project_name && pip install -e .
  !source activate $project_name && pip install light-the-torch
  !source activate $project_name && ltt install torch
  !source activate $project_name && pip3 install dgl==0.4.3 dglke
%cd $project_name

--2021-01-16 23:53:45--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2021-01-16 23:53:45--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   170MB/s    in 0.3s    

2021-01-16 23:53:45 (170 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_

In [8]:
import pandas as pd
import numpy as np
drkg_file = 'data/external/drkg/drkg.tsv'
df = pd.read_csv(drkg_file, sep="\t")
triples = df.values.tolist()
num_triples = len(triples)

seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
valid_cnt = int(num_triples * 0.05)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
valid_set = seed[train_cnt:train_cnt+valid_cnt].tolist()
test_set = seed[train_cnt+valid_cnt:].tolist()

with open("data/external/drkg/drkg_train.tsv", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
with open("data/external/drkg/drkg_valid.tsv", 'w+') as f:
    for idx in valid_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("data/external/drkg/drkg_test.tsv", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

      

In [ ]:
!source activate $project_name && DGLBACKEND=pytorch dglke_train --dataset ComplEx --data_path data/external/drkg \
--data_files drkg.tsv --format 'raw_udd_hrt' --model_name ComplEx --batch_size 2048 \
--neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 1000000 --log_interval 1000 \
--batch_size_eval 16 -adv --regularization_coef 1.00E-07 --num_thread 1 --gpu 0 --neg_sample_size_eval 10000 --mix_cpu_gpu

Reading train triples....
Finished. Read 5874261 train triples.
|Train|: 5874261
/usr/local/envs/google_colab/lib/python3.6/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
Total initialize time 20.832 seconds
[proc 0][Train](1000/1000000) average pos_loss: 0.4092985180020332
[proc 0][Train](1000/1000000) average neg_loss: 0.736289720416069
[proc 0][Train](1000/1000000) average loss: 0.5727941182553768
[proc 0][Train](1000/1000000) average regularization: 0.007224725496757401
[proc 0][Train] 1000 steps take 18.669 seconds
[proc 0]sample: 3.137, forward: 6.854, backward: 2.535, update: 6.134
[proc 0][Train](2000/1000000) average pos_loss: 0.3765637463927269
[proc 0][Train](2000/1000000) average neg_loss: 0.6044235309362411
[proc 0][Train](2000/1000000) average loss: 0.49049363839626314
[proc 0][Train](2000/1000000) average regularization: 0.011278071170672775
[proc 0][Train] 10

In [ ]:
!mv ckpts models/

In [11]:
# !source activate $project_name && DGLBACKEND=pytorch dglke_train --dataset ComplEx --data_path data/external/drkg \
# --data_files drkg_train.tsv drkg_valid.tsv drkg_test.tsv --format 'raw_udd_hrt' --model_name ComplEx --batch_size 2048  --test \
# --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 \
# --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --num_thread 1 --gpu 0 --neg_sample_size_eval 10000 --mix_cpu_gpu

In [ ]:
# [0]Test average MRR: 0.6097390566704133
# [0]Test average MR: 93.7949324680896
# [0]Test average HITS@1: 0.5217967880209592
# [0]Test average HITS@3: 0.6639508635981383
# [0]Test average HITS@10: 0.7669800110992704

In [ ]:
output_paths = ['models/ckpts/']

import zipfile
import os
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))

def zip(path_to_zip, zip_name):
    zipf = zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED)
    zipdir(path_to_zip, zipf)
    zipf.close()


output_files = []
for output_path in output_paths:
  output_file = output_path.replace('/','__')+'.zip'
  output_files.append(output_file)
  zip(output_path, output_file)
  !cd /content && cp -r $project_name/$output_file /content/drive/MyDrive/$project_name/$output_file

files_str = ' '.join(output_files)
print(f"python drive.py -p {files_str} -d")

In [40]:
# !ls
# !source activate $project_name && python src/colab.py

In [38]:
# [zipdir(folder) for folder in ['data','models']];
# print("Files zipped")

# %cd /content
# !cd /content && cp -r $project_name/data.zip /content/drive/MyDrive/$project_name/
# !cd /content && cp -r $project_name/models.zip /content/drive/MyDrive/$project_name/
# print("Files copied to google drive")
# drive.flush_and_unmount()


In [39]:
# !cd /content && cp -r $project_name/ckpts/ComplEx_DRKG_2/DRKG_ComplEx_entity.npy /content/drive/MyDrive/$project_name/